# ESN Experiment

This notebook reproduces the results in the paper ...

It consits of three parts:

1. An example for how to set up and experiment and use the code to train an ESN on the data.
2. A section to reproduce the scores mentioned in the paper for each testset.
3. A section to measure training time of an echo state network on our dataset.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/gesture-ip

!pip install reservoirpy
!pip install Levenshtein
!pip install bayesian-optimization
!pip install hyperopt

%load_ext autoreload
%autoreload 2

import DataSet
import Evaluation
import datetime
import numpy as np 
import json
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages

from Utils import *

from DataSet import UniHHIMUGestures
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
from bayes_opt import BayesianOptimization
from reservoirpy.nodes import Reservoir, Ridge, IPReservoir, FORCE, LMS, RLS, NVAR
from reservoirpy.mat_gen import bernoulli, normal, fast_spectral_initialization, uniform, random_sparse
from reservoirpy.hyper import research
from reservoirpy.utils import verbosity
verbosity(0)

Mounted at /content/drive
/content/drive/MyDrive/gesture-ip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 167 kB 15.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 13.3 MB/s 
     |████████████████████████████████| 2.0 MB 63.9 MB/s 
     |████████████████████████████████| 104 kB 25.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=3847e3affd96499b640a851af3776337b811cba1a34d17e6d4865331c5251a74
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels

0

In [8]:
def fix_seed(manualSeed):
    
    np.random.seed(manualSeed)
    random.seed(manualSeed)
    torch.manual_seed(manualSeed)
    # if you are using GPU
    torch.cuda.manual_seed(manualSeed)
    torch.cuda.manual_seed_all(manualSeed)

fix_seed(1)

#===========================================================================
# Give this run a name. 
# If name equals 'test', no log will be generated
#===========================================================================
name = 'test'


#===========================================================================
# Decide which gesture data shall be used for training
#===========================================================================
inputGestures = [0,1,2,3,4,5,6,7,8,9]

#===========================================================================
# Decide which target signals shall be used for training
#===========================================================================
usedGestures = [0,1,2,3,4,5,6,7,8,9]

#===========================================================================
# Concatenate data to create "more" training samples, 1 corresponds to no concatenations
#===========================================================================
concFactor = 1

#===========================================================================
# Add noise to the data, 0 corresponds to no noise. Noise above 2 has shown to weaken recognition
#===========================================================================
noiseFactor = 1

#===========================================================================
# Decide wether gestures shall be shuffled before training. If true, nFolds many 
# pieces will be generated. Not every piece is garanteed to contain every gesture, so do not use too many.
#===========================================================================
shuffle = True
nFolds = 4


#===========================================================================
# Function used to evaluate during cross validation. Possible functions are:
# Evaluation.calc1MinusF1FromMaxApp (best working, used in thesis)
# Oger.utils.nmse (normalised mean square error, tells nothing about classifier perfomance but works okay)
# Evaluation.calcLevenshteinError (use the Levenshtein error, disadvantages are highlighted in thesis) 
# Evaluation.calc1MinusF1FromInputSegment (use segmentation by supervised signal)
#===========================================================================
evaluationFunction = Evaluation.calc1MinusF1FromMaxApp

#===========================================================================
# Set this to true if another output neuron shall be added to represent "no gesture"
#===========================================================================
learnTreshold = False

#===========================================================================
# Use on of the optimisation dictionaries from the optDicts file
#===========================================================================
optDict = 'bestParas'

#===========================================================================
# Use normalizer
#===========================================================================
useNormalized = 2

#===========================================================================
# Pick datasets to train on, and datasets to test on
#===========================================================================
inputFiles = ['s','j','na','l']
testFiles = ['ni']

# If desired add a specific file to test on, e.g. randTestFiles = ['lana_0_0.npz']
randTestFiles = []



#===========================================================================
# Setup project directory
#===========================================================================
now = datetime.datetime.now()
resultsPath = getProjectPath()+'results/'
pdfFileName = now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.pdf'
pdfFilePath = resultsPath+'pdf/'+pdfFileName
npzFileName = now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.npz'
npzFilePath = resultsPath+'npz/'+npzFileName
bestFlowPath = resultsPath+'nodes/'+now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.p'
# pp = PdfPages(pdfFilePath)


#===========================================================================
# Add labels for gestures
#===========================================================================
totalGestureNames = ['left','right','forward','backward','bounce up','bounce down','turn left','turn right','shake lr','shake ud', \
                     'tap 1','tap 2','tap 3','tap 4','tap 5','tap 6','no gesture']
gestureNames = []
for i in usedGestures:
    gestureNames.append(totalGestureNames[i])
gestureNames.append('no gesture')

In [21]:
files = ['s','j','na','l','ni']

def createData(inputFiles, testFiles):
    trainset = UniHHIMUGestures(dataDir='dataSets/', 
                                train=True, 
                                inputFiles=inputFiles,
                                testFiles=testFiles,
                                useNormalized=useNormalized, 
                                learnTreshold=learnTreshold,
                                shuffle=True,
                               )

    testset = UniHHIMUGestures(dataDir='dataSets/', 
                               train=False, 
                               inputFiles=inputFiles,
                               testFiles=testFiles,
                               useNormalized=useNormalized, 
                               learnTreshold=learnTreshold,
                               shuffle=True
                              
                              )

    trainloader = DataLoader(trainset, batch_size=1,
                            shuffle=True, num_workers=1)
    testloader = DataLoader(testset, batch_size=1,
                            shuffle=True, num_workers=1)
    return trainset, testset, trainloader, testloader


def getData(trainloader):
    x = []
    y = []
    for inputs, targets in trainloader:
        x.append(inputs[0])
        y.append(targets[0])
    x = np.concatenate(x)
    y = np.concatenate(y)
    return (x, y)

def trainESN(trainloader, esn):
    x, y = getData(trainloader)
    return esn.fit(x, y, warmup=100)

def trainESN2(trainloader, esn):
    x, y = getData(trainloader)
    return esn.train(x, y)


# trainESN(trainloader, esn)

def createESN(units=500, lr=0.3, sr=1.25, ridge=1e-5):
    # reservoir = Reservoir(N=500, lr=0.3, sr=1.25)
    reservoir = Reservoir(units=int(units), lr=lr, sr=sr)
    # readout = Ridge(output_dim=10, ridge=1e-5)
    readout = Ridge(output_dim=10, ridge=ridge)
    esn = reservoir >> readout
    return esn

def createIPESN(N=500, lr=0.3, sr=1.25, iss=1, ridge=1e-5):
    # reservoir = Reservoir(N=500, lr=0.3, sr=1.25)
    reservoir = IPReservoir(units=int(N), lr=lr, sr=sr, input_scaling=iss)
    # readout = Ridge(output_dim=10, ridge=1e-5)
    readout = Ridge(output_dim=10, ridge=ridge)
    esn = reservoir >> readout
    return esn

def createESNDetailed(N, lr, sr, input_connectivity, rc_connectivity, fb_connectivity, ridge):
    reservoir = Reservoir(units=int(N), lr=lr, sr=sr, input_connectivity=input_connectivity, rc_connectivity=rc_connectivity, fb_connectivity=fb_connectivity)
    readout = Ridge(output_dim=10, ridge=ridge)
    esn = reservoir >> readout
    return esn

def createESNWeightOptimization(**params):
    readout = Ridge(output_dim=10, ridge=params['ridge'])
    schemes = {'0': bernoulli, '1': uniform, '2': normal}
    del params['ridge']
    winScheme = schemes[str(int(params['winScheme']))]
    del params['winScheme']
    wScheme = schemes[str(int(params['wScheme']))]
    del params['wScheme']
    wfbScheme = schemes[str(int(params['wfbScheme']))]
    del params['wfbScheme']
    biasScheme = schemes[str(int(params['biasScheme']))]
    del params['biasScheme']
    params['units'] = int(params['units'])
    reservoir = Reservoir(**params, Win=winScheme, W=wScheme, Wfb=wfbScheme, bias=biasScheme)
    esn = reservoir >> readout
    return esn

def createESNWBernouolli(**params):
    readout = Ridge(output_dim=10, ridge=params['ridge'])
    del params['ridge']
    params['units'] = int(params['units'])
    reservoir = Reservoir(**params, W=bernoulli)
    esn = reservoir >> readout
    return esn
    
def createESNWUniform(**params):
    readout = Ridge(output_dim=10, ridge=params['ridge'])
    del params['ridge']
    params['units'] = int(params['units'])
    reservoir = Reservoir(**params, W=uniform)
    esn = reservoir >> readout
    return esn

def createFeedbackESN(**params):
    readout = Ridge(output_dim=10, ridge=params['ridge'])
    del params['ridge']
    params['units'] = int(params['units'])
    reservoir = Reservoir(**params)
    reservoir <<= readout
    esn = reservoir >> readout
    return esn

def createIPESNDetailed(**params):
    readout = Ridge(output_dim=10, ridge=params['ridge'])
    del params['ridge']
    params['units'] = int(params['units'])
    reservoir = IPReservoir(**params)
    esn = reservoir >> readout
    return esn

def createESNForce(**params):
    readout = FORCE(output_dim=10, alpha=params['alpha'])
    del params['alpha']
    params['units'] = int(params['units'])
    reservoir = Reservoir(**params)
    esn = reservoir >> readout
    return esn

def createESNLms(**params):
    readout = LMS(output_dim=10, alpha=params['alpha'])
    del params['alpha']
    params['units'] = int(params['units'])
    reservoir = Reservoir(**params)
    esn = reservoir >> readout
    return esn
    
def createESNRls(**params):
    readout = RLS(output_dim=10, alpha=params['alpha'])
    del params['alpha']
    params['units'] = int(params['units'])
    reservoir = Reservoir(**params)
    esn = reservoir >> readout
    return esn

def createNVAR(**params):
    readout = Ridge(output_dim=10, ridge=params['ridge'])
    del params['ridge']
    nvar = NVAR(**params)
    model = nvar >> readout
    return model

import sklearn

def testESN(esn, testloader, learnTreshold, fixed_threshold=0.4, plot=False, conf_mat_title="", testFiles=""):
    
    testCms = []
    testF1MaxApps1 = []
    testAccuracies1 = []
    testF1MaxApps2 = []
    testAccuracies2 = []
    if testFiles:
        trainset, testset, trainloader, testloader2 = createData(inputFiles=inputFiles, testFiles=testFiles)
        inputs1 = np.array([])
        targets1 = np.array([])
        inputs2 = np.array([])
        targets2 = np.array([])
        for test_input, test_target in testloader2:
            inputs2 = np.append(inputs2, test_input)
            targets2 = np.append(targets2, test_target)
        for test_input, test_target in testloader:
            inputs1 = np.append(inputs1, test_input)
            targets1 = np.append(targets1, test_target)
        print(np.all(targets2==targets1))

    for test_inputs, test_targets in testloader:
        outputs = esn.run(test_inputs[0])
    
        t_target = test_targets[0].numpy()
        prediction = outputs
        if learnTreshold: # if threshold is learned, then it's the last collumn of the prediction
            threshold = outputs[0].numpy()[:,10]
        else: #else add a constant threshold
            threshold = np.ones((prediction.shape[0],1))*fixed_threshold

        t_maxApp_prediction = Evaluation.calcMaxActivityPrediction(prediction,t_target,threshold, 10)


        pred_MaxApp, targ_MaxApp = Evaluation.calcInputSegmentSeries(t_maxApp_prediction, t_target, 0.5)
        testF1MaxApps1.append(np.mean(sklearn.metrics.f1_score(targ_MaxApp,pred_MaxApp,average=None)))
        testAccuracies1.append(np.mean(sklearn.metrics.accuracy_score(targ_MaxApp,pred_MaxApp)))

    
    for test_inputs, test_targets in testloader2:
        outputs = esn.run(test_inputs[0])
    
        t_target = test_targets[0].numpy()
        prediction = outputs
        if learnTreshold: # if threshold is learned, then it's the last collumn of the prediction
            threshold = outputs[0].numpy()[:,10]
        else: #else add a constant threshold
            threshold = np.ones((prediction.shape[0],1))*fixed_threshold

        t_maxApp_prediction = Evaluation.calcMaxActivityPrediction(prediction,t_target,threshold, 10)


        pred_MaxApp, targ_MaxApp = Evaluation.calcInputSegmentSeries(t_maxApp_prediction, t_target, 0.5)
        testF1MaxApps2.append(np.mean(sklearn.metrics.f1_score(targ_MaxApp,pred_MaxApp,average=None)))
        testAccuracies2.append(np.mean(sklearn.metrics.accuracy_score(targ_MaxApp,pred_MaxApp)))

    print(testF1MaxApps1, testF1MaxApps2)
        
    return testF1MaxApps1, testAccuracies1, testCms

# f1scores, accuracies, testCms = testESN(esn, testloader, learnTreshold)

<h1>Utility For Optimization</h1>


In [22]:
def testModel(params, trainFiles, testFiles, numEvals=1, modelCreator=createESN, trainFunc=trainESN):
    scores = []
    train_scores = []
    accuracies = []
    networks = []
    cms = []
    for _ in range(numEvals):
        trainset, testset, trainloader, testloader = createData(inputFiles=trainFiles, testFiles=testFiles)
        esn = modelCreator(**params)
        try:
            trainFunc(trainloader, esn)
        except Exception as e:
            print(e)
            return 0, 0, 0, 0
        bestF1ScoreTreshold = 0.4
        # very good results when bestF1score is set to 0.4
        
        # train_score, train_accuracy, train_cms = testESN(esn, trainloader, learnTreshold, fixed_threshold=bestF1ScoreTreshold)
        score, accuracy, cm = testESN(esn, testloader, learnTreshold, fixed_threshold=bestF1ScoreTreshold, testFiles=testFiles)
        # train_scores.extend(train_score)
        scores.extend(score)
        accuracies.extend(accuracy)
        networks.append(esn)
        cms.extend(cm)
    return (np.array(scores).mean(), np.array(scores).std(), np.array(accuracies).mean(), np.array(accuracies).std())

def testOnAll(params, creator, trainer=trainESN):
    files = ['s','j','na','l','ni']
    f1Scores = []
    accuracies = []
    for idx in range(5):
        inputFiles = files[:idx] + files[idx+1:]
        testFiles = files[idx:idx+1]
        # print(inputFiles, testFiles)
        f1, f1std, accuracy, accuracyStd = testModel(params[idx], inputFiles, testFiles, 5, creator, trainer)
        print(f1, f1std, accuracy, accuracyStd)
        f1Scores.append(f1)
        accuracies.append(accuracy)
    print(np.array(f1Scores).mean(), np.array(f1Scores).std(), np.array(accuracies).mean(), np.array(accuracies).std())

<h1>Optimization Utility</h1>

In [23]:
def optimizer(pbounds, modelCreator, trainFunc, numEvals=3):
    files = ['s','j','na','l','ni']
    f1Scores = []
    accuracies = []
    for idx in range(5):
        inputFiles = files[:idx] + files[idx+1:]
        validationFiles = [inputFiles[idx%4]]
        trainFiles = inputFiles[:idx%4] + inputFiles[idx%4+1:]
        testFiles = files[idx:idx+1]

        def black_box_function(**params):
            f1, _, _, _ = testModel(params, trainFiles, validationFiles, numEvals, modelCreator, trainFunc)
            return f1

        optimizer = BayesianOptimization(
            f=black_box_function,
            pbounds=pbounds,
        )

        optimizer.maximize(
            init_points=15,
            n_iter=15,
        )
        f1, f1std, accuracy, accuracyStd = testModel(optimizer.max['params'], inputFiles, testFiles, 1, modelCreator, trainFunc)
        print(testFiles, f1, f1std, accuracy, accuracyStd, optimizer.max['params'])
        f1Scores.append(f1)
        accuracies.append(accuracy)
    print(np.array(f1Scores).mean(), np.array(f1Scores).std(), np.array(accuracies).mean(), np.array(accuracies).std())

In [25]:
# without create new loaders in testESN

params = [
    {'lr': 0.3, 'ridge': 1e-05, 'sr': 1.25, 'units': 500},
    {'lr': 0.3, 'ridge': 1e-05, 'sr': 1.25, 'units': 500},
    {'lr': 0.3, 'ridge': 1e-05, 'sr': 1.25, 'units': 500},
    {'lr': 0.3, 'ridge': 1e-05, 'sr': 1.25, 'units': 500},
    {'lr': 0.3, 'ridge': 1e-05, 'sr': 1.25, 'units': 500},
]
testOnAll(params, createESN)

['s']
False
[0.8220105333925326] [0.8202659301265501]
['s']
False
[0.891575815488859] [0.8172905525846702]
['s']
False
[0.8770212390143858] [0.8878096224939841]
['s']
False
[0.749998777857217] [0.7802179616063168]
['s']
False
[0.8521280951902963] [0.9022096270510712]
0.8385468921886581 0.05018169159943661 0.9099439052023148 0.02272783057324068
['j']
False
[0.8469326276812909] [0.8526396690012251]
['j']
False
[0.8771842687368178] [0.8770770117333678]
['j']
False
[0.9199539710375624] [0.9257137297064701]
['j']
False
[0.8709682345518259] [0.8983885644502704]
['j']
False
[0.871234976666394] [0.8919671430507343]
0.8772548157347781 0.02374201968738274 0.9318760422629714 0.01541269404810773
['na']
False
[0.742201853586041] [0.8873806093365556]
['na']
False
[0.7445475948825231] [0.9500214483988149]
['na']
False
[0.7419886614933054] [0.9092141510801799]
['na']
False
[0.7502951593860684] [0.8931292395893652]
['na']
False
[0.731279298319253] [0.9455173423536714]
0.7420625135334382 0.0061685441909

In [ ]:
# without create new loaders in testESN

params = [
    {'lr': 0.8156702597852322, 'ridge': 1.5638924098699497e-06, 'sr': 1.1395898362204437, 'units': 1079.1906513939543},
    {'lr': 0.2860986821992064, 'ridge': 1.6924807164555154e-05, 'sr': 1.08205922917847, 'units': 1702.6860361626098},
    {'lr': 0.7174510506956969, 'ridge': 5.28904511653844e-05, 'sr': 1.2460288451135533, 'units': 822.4117034484156},
    {'lr': 0.89214199596648, 'ridge': 6.738247536968547e-05, 'sr': 1.0766271510075016, 'units': 676.9140429974445},
    {'lr': 0.6679611858826577, 'ridge': 5.122782710362848e-06, 'sr': 1.3203967948530762, 'units': 1982.8666350494152}
]
testOnAll(params, createESN)

0.8856165706926922 0.04957524210305308 0.9365040682069823 0.02362237341203318
0.8710838646012748 0.04836083146868316 0.920378977965712 0.027233147839988574
0.7370179784479273 0.02439436827599421 0.8523912546123384 0.013679515419085173
0.8253068605700685 0.04953654399430559 0.8914062936582041 0.02401166158014276
0.8861907575909388 0.027069604814532015 0.9333279304939296 0.016914655999259207
0.8410432063805804 0.056562192246470085 0.9068017049874333 0.031520562057143035


In [ ]:
# with create new loaders in testESN

params = [
    {'lr': 0.8156702597852322, 'ridge': 1.5638924098699497e-06, 'sr': 1.1395898362204437, 'units': 1079.1906513939543},
    {'lr': 0.2860986821992064, 'ridge': 1.6924807164555154e-05, 'sr': 1.08205922917847, 'units': 1702.6860361626098},
    {'lr': 0.7174510506956969, 'ridge': 5.28904511653844e-05, 'sr': 1.2460288451135533, 'units': 822.4117034484156},
    {'lr': 0.89214199596648, 'ridge': 6.738247536968547e-05, 'sr': 1.0766271510075016, 'units': 676.9140429974445},
    {'lr': 0.6679611858826577, 'ridge': 5.122782710362848e-06, 'sr': 1.3203967948530762, 'units': 1982.8666350494152}
]
testOnAll(params, createESN)

['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array([0., 1.]), array([71078,  2112]))
['s']
['s']
(array([0., 1.]), array([71078,  2112]))
(array(

In [ ]:
# Fresh experiments
pbounds = {'units': (400, 2000), 'lr': (0, 1), 'sr': (0.5, 2), 'ridge': (1e-6, 1e-4)}
optimizer(pbounds, createESN, trainESN, 1)



|   iter    |  target   |    lr     |   ridge   |    sr     |   units   |
-------------------------------------------------------------------------
|  1        |  0.7033   |  0.2589   |  6.645e-0 |  1.585    |  1.147e+0 |
|  2        |  0.8521   |  0.4934   |  4.445e-0 |  0.5942   |  1.137e+0 |
|  3        |  0.1617   |  0.6959   |  2.948e-0 |  1.874    |  497.4    |
|  4        |  0.8608   |  0.3731   |  5.674e-0 |  1.434    |  1.179e+0 |
|  5        |  0.8623   |  0.7974   |  6.125e-0 |  1.072    |  1.667e+0 |
|  6        |  0.8659   |  0.1887   |  2.95e-05 |  1.059    |  617.2    |
|  7        |  0.8057   |  0.2111   |  7.734e-0 |  1.653    |  545.5    |
|  8        |  0.07754  |  0.02368  |  5.419e-0 |  1.323    |  1.838e+0 |
|  9        |  0.6597   |  0.1472   |  1.336e-0 |  0.9015   |  1.592e+0 |
|  10       |  0.4244   |  0.1994   |  7.446e-0 |  1.564    |  545.9    |
|  11       |  0.8465   |  0.7185   |  4.373e-0 |  1.397    |  1.85e+03 |
|  12       |  0.8831   |  0.7194   | 

TypeError: ignored

In [ ]:
files = ['s','j','na','l','ni']
f1Scores = []
accuracies = []
for idx in range(5):
    inputFiles = files[:idx] + files[idx+1:]
    testFiles = files[idx:idx+1]
    print(inputFiles, testFiles)
    f1, f1std, accuracy, accuracyStd = testModel({}, inputFiles, testFiles, 10, createESN, trainESN)
    print(f1, f1std, accuracy, accuracyStd)
    f1Scores.append(f1)
    accuracies.append(accuracy)
print(np.array(f1Scores).mean(), np.array(f1Scores).std(), np.array(accuracies).mean(), np.array(accuracies).std())

['j', 'na', 'l', 'ni'] ['s']
0.8578938236806708 0.03977109200724788 0.9244618491699915 0.0189398992496336
['s', 'na', 'l', 'ni'] ['j']
0.9077036146769732 0.03298187434777397 0.9480816756593752 0.018250657436617006
['s', 'j', 'l', 'ni'] ['na']
0.909498896992029 0.030639130215551837 0.9372632182514566 0.021630185888665023
['s', 'j', 'na', 'ni'] ['l']
0.857476276142253 0.05378011920818379 0.9279906192589701 0.024645688774485933
['s', 'j', 'na', 'l'] ['ni']
0.8760726098798852 0.039105698252752445 0.9302931291954714 0.020496117241955866
0.8817290442743623 0.022952789723429744 0.933618098307053 0.008355151794255082


In [ ]:
# ESN with uniform weight initializations hyperparam optimization

pbounds = {'units': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'ridge': (0, 1e-4)}
optimizer(pbounds, createESNWUniform, trainESN, 3)

|   iter    |  target   |    lr     |   ridge   |    sr     |   units   |
-------------------------------------------------------------------------
|  1        |  0.7933   |  0.5225   |  6.612e-0 |  0.6703   |  248.8    |
|  2        |  0.7137   |  0.9482   |  4.084e-0 |  1.406    |  898.6    |
|  3        |  0.5314   |  0.1195   |  3.505e-0 |  1.432    |  682.4    |
|  4        |  0.7632   |  0.3862   |  8.97e-05 |  0.5184   |  208.3    |
|  5        |  0.6177   |  0.1646   |  6.95e-05 |  1.591    |  413.5    |
|  6        |  0.806    |  0.9635   |  9.09e-05 |  0.6992   |  530.4    |
|  7        |  0.3158   |  0.3732   |  1.795e-0 |  1.754    |  643.1    |
|  8        |  0.8927   |  0.448    |  2.478e-0 |  0.9252   |  438.4    |
|  9        |  0.483    |  0.6344   |  8.881e-0 |  1.875    |  118.8    |
|  10       |  0.8355   |  0.14     |  6.017e-0 |  0.5529   |  556.6    |
|  11       |  0.8876   |  0.9243   |  5.5e-05  |  0.8006   |  706.6    |
|  12       |  0.8715   |  0.7753   | 

In [ ]:
# ESN with bernoulli weight initializations hyperparam optimization

pbounds = {'units': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'ridge': (0, 1e-4)}
optimizer(pbounds, createESNWBernouolli, trainESN, 3)

|   iter    |  target   |    lr     |   ridge   |    sr     |   units   |
-------------------------------------------------------------------------
|  1        |  0.1582   |  0.5781   |  2.376e-0 |  1.858    |  696.0    |
|  2        |  0.9181   |  0.6942   |  8.333e-0 |  1.237    |  611.9    |
|  3        |  0.7983   |  0.7596   |  9.897e-0 |  0.855    |  338.9    |
|  4        |  0.8935   |  0.9194   |  7.544e-0 |  0.8472   |  841.0    |
|  5        |  0.6069   |  0.2109   |  3.997e-0 |  1.549    |  355.0    |
|  6        |  0.8337   |  0.6484   |  3.546e-0 |  0.7171   |  213.4    |
|  7        |  0.856    |  0.3216   |  4.066e-0 |  1.182    |  577.2    |
|  8        |  0.8262   |  0.1376   |  4.23e-05 |  1.006    |  566.7    |
|  9        |  0.7958   |  0.1607   |  9.24e-05 |  1.356    |  176.2    |
|  10       |  0.8932   |  0.4384   |  2.815e-0 |  0.8992   |  997.0    |
|  11       |  0.1939   |  0.02066  |  7.772e-0 |  1.109    |  928.2    |
|  12       |  0.8787   |  0.5686   | 

In [ ]:
# ESN with different weight initializations hyperparam optimization

pbounds = {'units': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'ridge': (0, 1e-4), 'wScheme': (0, 2.99), 'winScheme': (0, 2.99), 'wfbScheme': (0, 2.99), 'biasScheme': (0, 2.99)}
optimizer(pbounds, createESNWeightOptimization, trainESN, 3)

|   iter    |  target   | biasSc... |    lr     |   ridge   |    sr     |   units   |  wScheme  | wfbScheme | winScheme |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6467   |  0.09975  |  0.2941   |  1.762e-0 |  1.101    |  112.2    |  2.703    |  2.749    |  0.144    |
|  2        |  0.841    |  1.944    |  0.279    |  2.077e-0 |  0.5384   |  419.6    |  2.569    |  0.6721   |  1.712    |
|  3        |  0.5707   |  2.133    |  0.7502   |  3.657e-0 |  1.524    |  184.8    |  2.895    |  1.432    |  2.333    |
|  4        |  0.8257   |  2.873    |  0.4557   |  4.243e-0 |  1.368    |  912.4    |  2.301    |  1.99     |  1.655    |
|  5        |  0.3226   |  0.6067   |  0.01491  |  6.995e-0 |  0.7501   |  633.2    |  0.6423   |  0.05221  |  2.458    |
|  6        |  0.8789   |  0.2402   |  0.2852   |  7.081e-0 |  1.185    |  841.8    |  1.765    |  1.901    |  1.26     |
|  7        |  0.8781   

In [ ]:
# ESN with feedback hyperparam optimization

pbounds = {'units': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'ridge': (0, 1e-4)}
optimizer(pbounds, createFeedbackESN, trainESN, 3)

|   iter    |  target   |    lr     |   ridge   |    sr     |   units   |
-------------------------------------------------------------------------
|  1        |  0.0822   |  0.7195   |  2.367e-0 |  1.347    |  852.5    |
|  2        |  0.07754  |  0.418    |  4.838e-0 |  1.001    |  789.3    |
|  3        |  0.1252   |  0.4465   |  7.985e-0 |  1.865    |  220.7    |
|  4        |  0.07754  |  0.1333   |  1.054e-0 |  1.176    |  260.2    |
|  5        |  0.1609   |  0.5325   |  4.113e-0 |  1.914    |  425.6    |
|  6        |  0.07754  |  0.1415   |  8.071e-0 |  0.9738   |  246.5    |
|  7        |  0.08686  |  0.4528   |  2.811e-0 |  1.056    |  555.6    |
|  8        |  0.1219   |  0.1866   |  3.267e-0 |  1.987    |  630.4    |
|  9        |  0.1108   |  0.8648   |  3.527e-0 |  1.602    |  770.2    |
|  10       |  0.08312  |  0.929    |  2.904e-0 |  1.472    |  221.1    |


KeyboardInterrupt: ignored

In [ ]:
# ESN_RLS hyperparam optimization

pbounds = {'units': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'alpha': (1e-5, 1e-7)}
optimizer(pbounds, createESNRls, trainESN2, 1)

|   iter    |  target   |   alpha   |    lr     |    sr     |   units   |
-------------------------------------------------------------------------
|  1        |  0.5936   |  7.219e-0 |  0.0601   |  0.5062   |  871.3    |
|  2        |  0.7256   |  7.364e-0 |  0.7138   |  0.921    |  176.9    |
|  3        |  0.8422   |  2.005e-0 |  0.8716   |  0.5877   |  828.6    |
|  4        |  0.1647   |  6.784e-0 |  0.9863   |  1.93     |  433.3    |
|  5        |  0.9133   |  7.925e-0 |  0.6463   |  1.122    |  471.5    |
|  6        |  0.8237   |  8.028e-0 |  0.1056   |  1.642    |  486.4    |
|  7        |  0.8648   |  5.363e-0 |  0.1407   |  0.6095   |  264.7    |
|  8        |  0.7505   |  1.924e-0 |  0.1054   |  1.176    |  626.4    |
|  9        |  0.7253   |  1.333e-0 |  0.5384   |  1.458    |  852.0    |
|  10       |  0.2488   |  1.31e-06 |  0.5316   |  1.832    |  905.0    |
|  11       |  0.8519   |  8.837e-0 |  0.7146   |  1.548    |  655.1    |
|  12       |  0.9101   |  6.676e-0 | 

In [ ]:
# ESN_LMS hyperparam optimization

pbounds = {'units': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'alpha': (1e-6, 1e-6)}
optimizer(pbounds, createESNLms, trainESN2, 1)

|   iter    |  target   |   alpha   |    lr     |    sr     |   units   |
-------------------------------------------------------------------------
|  1        |  0.06081  |  1e-06    |  0.5108   |  1.686    |  880.2    |
|  2        |  0.06081  |  1e-06    |  0.4957   |  0.8738   |  974.8    |
|  3        |  0.06081  |  1e-06    |  0.661    |  1.28     |  858.4    |
|  4        |  0.06081  |  1e-06    |  0.5285   |  1.149    |  984.1    |
|  5        |  0.06081  |  1e-06    |  0.6154   |  0.7274   |  952.4    |
|  6        |  0.06081  |  1e-06    |  0.1312   |  0.567    |  535.8    |
|  7        |  0.06081  |  1e-06    |  0.2039   |  0.7135   |  697.2    |
|  8        |  0.06081  |  1e-06    |  0.1878   |  1.551    |  129.6    |
|  9        |  0.06081  |  1e-06    |  0.5731   |  1.104    |  405.0    |
|  10       |  0.06081  |  1e-06    |  0.6923   |  1.638    |  990.0    |
|  11       |  0.06081  |  1e-06    |  0.6853   |  1.468    |  165.1    |
|  12       |  0.06081  |  1e-06    | 

KeyboardInterrupt: ignored

In [ ]:
# ESN_Force hyperparam optimization

pbounds = {'units': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'alpha': (1e-5, 1e-7)}
optimizer(pbounds, createESNForce, trainESN2, 1)

|   iter    |  target   |   alpha   |    lr     |    sr     |   units   |
-------------------------------------------------------------------------
|  1        |  0.5002   |  9.959e-0 |  0.02193  |  0.8226   |  255.6    |
|  2        |  0.8467   |  5.25e-06 |  0.9117   |  0.8494   |  648.3    |
|  3        |  0.8287   |  7.438e-0 |  0.9943   |  0.6729   |  903.2    |
|  4        |  0.7908   |  5.919e-0 |  0.4372   |  0.7541   |  271.9    |
|  5        |  0.1925   |  3.872e-0 |  0.1256   |  1.9      |  684.1    |
|  6        |  0.8132   |  2.302e-0 |  0.8938   |  0.9401   |  436.4    |
|  7        |  0.9222   |  2.528e-0 |  0.3199   |  0.7973   |  375.4    |
|  8        |  0.7459   |  2.789e-0 |  0.7775   |  0.8434   |  256.6    |
|  9        |  0.8481   |  4.127e-0 |  0.3146   |  0.995    |  934.9    |
|  10       |  0.7487   |  2.351e-0 |  0.8419   |  0.8855   |  250.5    |
|  11       |  0.8942   |  1.248e-0 |  0.7154   |  1.402    |  594.3    |
|  12       |  0.8302   |  8.005e-0 | 

In [ ]:
# detailed IP-ESN hyperparam optimization

pbounds = {'units': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'mu': (-0.2, 0.2), 'sigma': (0, 2), 'learning_rate': (1e-2, 1e-5), 'epochs': (3, 3), 'input_connectivity': (0, 0.2), 'rc_connectivity': (0, 0.2), 'fb_connectivity': (0, 0.2), 'ridge': (0, 1e-4)}
optimizer(pbounds, createIPESNDetailed)

|   iter    |  target   |  epochs   | fb_con... | input_... | learni... |    lr     |    mu     | rc_con... |   ridge   |   sigma   |    sr     |   units   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.2447   |  3.0      |  0.04275  |  0.172    |  4.042e-0 |  0.377    |  0.1564   |  0.08209  |  9.215e-0 |  0.9901   |  1.793    |  412.9    |
|  2        |  0.7345   |  3.0      |  0.1768   |  0.1546   |  0.007762 |  0.2003   | -0.03251  |  0.04855  |  2.431e-0 |  0.3312   |  0.5484   |  729.6    |
|  3        |  0.6278   |  3.0      |  0.06114  |  0.1008   |  0.002519 |  0.9907   |  0.141    |  0.189    |  7.907e-0 |  0.4801   |  0.5827   |  606.2    |
|  4        |  0.2804   |  3.0      |  0.1424   |  0.143    |  0.000332 |  0.4773   |  0.0245   |  0.08396  |  4.232e-0 |  0.806    |  1.338    |  850.5    |
|  5        |  0.3742   |  3.0      |  0.1139   |  0

In [ ]:
# detailed ESN hyperparam optimization

pbounds = {'N': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'input_connectivity': (0, 0.2), 'rc_connectivity': (0, 0.2), 'fb_connectivity': (0, 0.2), 'ridge': (0, 1e-4)}
optimizer(pbounds, createESNDetailed)

|   iter    |  target   |     N     | fb_con... | input_... |    lr     | rc_con... |   ridge   |    sr     |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.2672   |  369.0    |  0.04314  |  0.006295 |  0.1352   |  0.09758  |  5.898e-0 |  1.902    |
|  2        |  0.901    |  837.9    |  0.07493  |  0.0604   |  0.6829   |  0.1148   |  6.238e-0 |  0.9758   |
|  3        |  0.8973   |  908.3    |  0.1221   |  0.108    |  0.7277   |  0.1965   |  2.575e-0 |  0.7736   |
|  4        |  0.3299   |  190.4    |  0.173    |  0.06144  |  0.3534   |  0.196    |  6.96e-05 |  1.94     |
|  5        |  0.6579   |  185.2    |  0.1111   |  0.01947  |  0.1847   |  0.131    |  1.454e-0 |  0.9358   |
|  6        |  0.9142   |  703.8    |  0.141    |  0.1959   |  0.48     |  0.1914   |  1.946e-0 |  1.17     |
|  7        |  0.6927   |  346.4    |  0.1271   |  0.0936   |  0.9295   |  0.05331  |  1.398e-0 |  1.036    |
|  8      

<h1>Bayesian Optimization</h1>

In [ ]:
def cvBayesian():
    files = ['s','j','na','l','ni']
    f1Scores = []
    accuracies = []
    for idx in range(5):
        inputFiles = files[:idx] + files[idx+1:]
        validationFiles = [inputFiles[idx%4]]
        trainFiles = inputFiles[:idx%4] + inputFiles[idx%4+1:]
        testFiles = files[idx:idx+1]
        print(trainFiles, validationFiles, testFiles)

        def black_box_function(N, lr, sr, iss, ridge):
            f1, _, _, _ = testModel({'N': N, 'lr': lr, 'sr': sr, 'iss': iss, 'ridge': ridge}, trainFiles, validationFiles, 2, createESN)
            return f1

        pbounds = {'N': (100, 1000), 'lr': (0, 1), 'sr': (0.5, 2), 'iss': (0.9, 1.1), 'ridge': (0, 1e-4)}

        optimizer = BayesianOptimization(
            f=black_box_function,
            pbounds=pbounds,
        )

        optimizer.maximize(
            init_points=15,
            n_iter=15,
        )
        f1, f1std, accuracy, accuracyStd = testModel(optimizer.max['params'], inputFiles, testFiles, 10, createESN)
        print(f1, optimizer.max['params'])
        f1Scores.append(f1)
        accuracies.append(accuracy)
    print(np.array(f1Scores).mean(), np.array(accuracies).mean())
cvBayesian()

['na', 'l', 'ni'] ['j'] ['s']
|   iter    |  target   |     N     |    iss    |    lr     |   ridge   |    sr     |
-------------------------------------------------------------------------------------
|  1        |  0.2068   |  946.8    |  0.9549   |  0.2139   |  7.559e-0 |  1.737    |
|  2        |  0.8974   |  930.7    |  0.9163   |  0.825    |  3.486e-0 |  1.336    |
|  3        |  0.837    |  215.9    |  1.007    |  0.5596   |  6.848e-0 |  0.921    |
|  4        |  0.3255   |  259.0    |  1.059    |  0.2912   |  5.601e-0 |  1.601    |
|  5        |  0.8915   |  497.0    |  1.093    |  0.3639   |  5.809e-0 |  1.182    |
|  6        |  0.897    |  519.1    |  0.9301   |  0.2653   |  5.886e-0 |  1.657    |
|  7        |  0.895    |  257.3    |  1.076    |  0.4688   |  7.264e-0 |  1.467    |
|  8        |  0.848    |  867.4    |  0.9034   |  0.9928   |  6.646e-0 |  1.392    |
|  9        |  0.1379   |  214.4    |  0.9357   |  0.8319   |  8.61e-05 |  1.92     |
|  10       |  0.8233   

<h1>HyperOpt</h1>

In [ ]:
from reservoirpy.observables import nrmse, rsquare

def objective(dataset, config, *, iss, N, sr, lr, ridge, seed):
    # This step may vary depending on what you put inside 'dataset'
    trainloader, testloader = dataset
    testDataset = getData(testloader)
    trainDataset = getData(trainloader)
    X_train, y_train = trainDataset
    X_val, y_val = testDataset

    # You can access anything you put in the config
    # file from the 'config' parameter.
    instances = config["instances_per_trial"]

    # The seed should be changed across the instances,
    # to be sure there is no bias in the results
    # due to initialization.
    variable_seed = seed

    losses = []; r2s = [];
    for n in range(instances):
        model = createESN(N, lr, sr, iss, ridge)

        model = trainESN(trainloader, model)

        # Train your model and test your model.
        predictions = model.run(X_val)

        loss = nrmse(y_val, predictions, norm_value=np.ptp(X_train))
        r2 = rsquare(y_val, predictions)

        # Change the seed between instances
        variable_seed += 1

        losses.append(loss)
        r2s.append(r2)

    # Return a dictionnary of metrics. The 'loss' key is mandatory when
    # using hyperopt.
    return {'loss': np.mean(losses),
            'r2': np.mean(r2s)}

def cvHyperOpt():
    files = ['s','j','na','l','ni']
    f1Scores = []
    accuracies = []
    for idx in range(5):
        inputFiles = files[:idx] + files[idx+1:]
        validationFiles = [inputFiles[idx%4]]
        trainFiles = inputFiles[:idx%4] + inputFiles[idx%4+1:]
        testFiles = files[idx:idx+1]
        print(trainFiles, validationFiles, testFiles)
        trainset, testset, trainloader, valloader = createData(inputFiles=trainFiles, testFiles=validationFiles)

        hyperopt_config = {
            "exp": f"hyperopt-"+str(idx),  # the experimentation name
            "hp_max_evals": 25,             # the number of differents sets of parameters hyperopt has to try
            "hp_method": "random",           # the method used by hyperopt to chose those sets (see below)
            "seed": 42,                      # the random state seed, to ensure reproducibility
            "instances_per_trial": 2,        # how many random ESN will be tried with each sets of parameters
            "hp_space": {                    # what are the ranges of parameters explored
                "N": ["loguniform", 100, 1000],             # the number of neurons is fixed to 300
                "sr": ["loguniform", 1, 1.5],   # the spectral radius is log-uniformly distributed between 1e-6 and 10
                "lr": ["loguniform", 0.1, 0.5],  # idem with the leaking rate, from 1e-3 to 1
                "iss": ["loguniform", 0.8, 1.2],           # the input scaling is fixed
                "ridge": ["loguniform", 0.1, 1e-4],        # and so is the regularization parameter.
                "seed": ["choice", 1234]          # an other random seed for the ESN initialization
            }
        }

        # we precautionously save the configuration in a JSON file
        # each file will begin with a number corresponding to the current experimentation run number.
        with open(f"{hyperopt_config['exp']}.config.json", "w+") as f:
            json.dump(hyperopt_config, f)

        best = research(objective, (trainloader, valloader), f"{hyperopt_config['exp']}.config.json", ".")
        bestParams = best[0]
        del bestParams["seed"]
        f1, f1std, accuracy, accuracyStd = testModel(bestParams, inputFiles, testFiles, 10)
        print(f1, bestParams)
        f1Scores.append(f1)
        accuracies.append(accuracy)
    print(np.array(f1Scores).mean(), np.array(accuracies).mean())
cvHyperOpt()

['na', 'l', 'ni'] ['j'] ['s']
100%|██████████| 25/25 [09:04<00:00, 21.78s/it, best loss: 0.07915491298478249]
0.8077018746476494 {'N': 134.37380066287758, 'iss': 0.9421461092028736, 'lr': 0.13977183932409387, 'ridge': 0.03495943578044508, 'sr': 1.2686163121459586}
['s', 'l', 'ni'] ['na'] ['j']
100%|██████████| 25/25 [08:48<00:00, 21.13s/it, best loss: 0.07829489073336328]
0.9029965608570005 {'N': 688.2413605222424, 'iss': 0.8250358321874488, 'lr': 0.2849384015301757, 'ridge': 0.0026730042614421198, 'sr': 1.206071841624978}
['s', 'j', 'ni'] ['l'] ['na']
100%|██████████| 25/25 [07:57<00:00, 19.12s/it, best loss: 0.06208555014456327]
0.8617891181505304 {'N': 873.553735349567, 'iss': 0.9660075849286349, 'lr': 0.43026659367097475, 'ridge': 0.05422990705217484, 'sr': 1.3781881151416353}
['s', 'j', 'na'] ['ni'] ['l']
100%|██████████| 25/25 [08:47<00:00, 21.11s/it, best loss: 0.08388823623763046]
0.7149257226852248 {'N': 159.16141529654945, 'iss': 0.9909281373838861, 'lr': 0.32432429020873504,

In [ ]:
testModel({'N': 801.1190927422981, 'iss': 1.0, 'lr': 0.4309931697316194, 'ridge': 9.54861694436978e-05, 'sr': 1.123707385863479}, ['j', 'na', 'l', 'ni'], ['s'])

['j', 'na', 'l', 'ni'] ['s']
['j', 'na', 'l', 'ni'] ['s']
['j', 'na', 'l', 'ni'] ['s']
['j', 'na', 'l', 'ni'] ['s']


(0.8314158685621362,
 0.02354094301034373,
 0.9121061359867331,
 0.015379135150075777)

In [ ]:
fix_seed(1)

files = ['s','j','na','l','ni']

all_scores = []
all_train_scores = []
all_networks = []
all_accuracies = []
all_cms = []

for idx in range(5):
    scores = []
    train_scores = []
    accuracies = []
    networks = []
    cms = []
    
    # Shuffle testsets
    inputFiles = files[:idx] + files[idx+1:]
    testFiles = files[idx:idx+1]
    print(inputFiles, testFiles)
    params = {'N': 500, 'lr': 0.3, 'ridge': 1e-05, 'sr': 1.25}


    for _ in range(2):
        trainset, testset, trainloader, testloader = createData(inputFiles=inputFiles, testFiles=testFiles)
        esn = createIPESN(**params)
        trainESN(trainloader, esn)
        
        if False:
            totalTrainInputData = []
            totalTrainTargetData = []
            for inputs, targets in trainloader:
                totalTrainInputData.append(inputs)
                totalTrainTargetData.append(targets)
            totalTrainInputData = torch.cat(totalTrainInputData,1)
            totalTrainTargetData = torch.cat(totalTrainTargetData,1)
            totalTrainInputData = torch.tensor(totalTrainInputData)
            totalTrainPrediction = esn(totalTrainInputData.float())

            totalTrainPrediction = totalTrainPrediction[0,:,:].numpy()
            totalTrainTargetData = totalTrainTargetData[0,:,:].numpy()
        
            tresholds, _, bestF1ScoreTreshold = Evaluation.calcTPFPForThresholds(
                totalTrainPrediction, totalTrainTargetData, 'Train Data Confusion - Target Treshold', False, plot=False)
        bestF1ScoreTreshold = 0.4
        # very good results when bestF1score is set to 0.4
        
        train_score, train_accuracy, train_cms = testESN(esn, trainloader, learnTreshold, 
                              fixed_threshold=bestF1ScoreTreshold)
        score, accuracy, cm = testESN(esn, testloader, learnTreshold, 
                        fixed_threshold=bestF1ScoreTreshold, testFiles=testFiles)
        train_scores.extend(train_score)
        scores.extend(score)
        print(score)
        accuracies.extend(accuracy)
        networks.append(esn)
        cms.extend(cm)
    print(np.array(scores).mean())
    print('{}: avg f1 score: {:.2f} ({:.2f}) '.format(testFiles[0], np.array(scores).mean(),np.array(scores).std()))
    all_scores.append(scores)
    all_train_scores.append(train_scores)
    all_accuracies.append(accuracies)
    all_networks.append(networks)
    all_cms.append(cms)

['j', 'na', 'l', 'ni'] ['s']
[0.25859085529482795]
[0.29953512814293803]
0.279062991718883
s: avg f1 score: 0.28 (0.02) 
['s', 'na', 'l', 'ni'] ['j']


KeyboardInterrupt: ignored

In [ ]:
# reservoir = Reservoir(lr=0.8989990333315352, sr=1.098822297046856, units=380)
# readout = Ridge(output_dim=10, ridge=4.522194488074985e-05)
# esn = reservoir >> readout

print("""Total results: 
    Avg train score: {:.2f} ({:.2f}), 
    avg test score: {:.2f} ({:.2f}), 
    acc: {:.2f} ({:.2f})""".format(
    np.array(all_train_scores).mean(), 
    np.array(all_train_scores).std(), 
    np.array(all_scores).mean(),
    np.array(all_scores).std(), 
    np.array(accuracies).mean(), 
    np.array(accuracies).std(),
))



results = np.hstack([
        np.array(all_train_scores).mean(1, keepdims=True), 
        np.array(all_train_scores).std(1, keepdims=True), 
        np.array(all_scores).mean(1, keepdims=True),
        np.array(all_scores).std(1, keepdims=True), 
        np.array(all_accuracies).mean(1, keepdims=True),
])

for file, result in zip(files, results):
    print("{}: Avg train score: {:.2f} ({:.2f}), avg test score: {:.2f} ({:.2f}) acc: {:.2f}".format(file, *result))
    


In [ ]:
# reservoir = Reservoir(units=500, lr=0.3, sr=1.25)
# readout = Ridge(output_dim=10, ridge=1e-5)
# esn = reservoir >> readout

print("""Total results: 
    Avg train score: {:.2f} ({:.2f}), 
    avg test score: {:.2f} ({:.2f}), 
    acc: {:.2f} ({:.2f})""".format(
    np.array(all_train_scores).mean(), 
    np.array(all_train_scores).std(), 
    np.array(all_scores).mean(),
    np.array(all_scores).std(), 
    np.array(accuracies).mean(), 
    np.array(accuracies).std(),
))



results = np.hstack([
        np.array(all_train_scores).mean(1, keepdims=True), 
        np.array(all_train_scores).std(1, keepdims=True), 
        np.array(all_scores).mean(1, keepdims=True),
        np.array(all_scores).std(1, keepdims=True), 
        np.array(all_accuracies).mean(1, keepdims=True),
])

for file, result in zip(files, results):
    print("{}: Avg train score: {:.2f} ({:.2f}), avg test score: {:.2f} ({:.2f}) acc: {:.2f}".format(file, *result))
    


Total results: 
    Avg train score: 0.97 (0.01), 
    avg test score: 0.87 (0.05), 
    acc: 0.95 (0.00)
s: Avg train score: 0.97 (0.01), avg test score: 0.79 (0.02) acc: 0.89
j: Avg train score: 0.96 (0.01), avg test score: 0.88 (0.02) acc: 0.94
na: Avg train score: 0.98 (0.01), avg test score: 0.92 (0.00) acc: 0.95
l: Avg train score: 0.95 (0.01), avg test score: 0.84 (0.04) acc: 0.92
ni: Avg train score: 0.97 (0.01), avg test score: 0.92 (0.01) acc: 0.95


In [ ]:
(np.mean([0.624 - 0.62, 0.791 - 0.79, 0.809 - 0.78, 0.831 - 0.76, 0.926 - 0.88]),
np.mean([0.624 - 0.60, 0.791 - 0.79, 0.809 - 0.80, 0.831 - 0.76, 0.926 - 0.86]),)


print("paper mean:           {:.3f}".format(np.mean([0.624,0.791,0.809,0.831,0.926])))

print("paper reproduct mean: {:.3f}".format(np.mean([0.59,0.74,0.77,0.79,0.86])))
print("opti. threshold mean: {:.3f}".format(np.mean([0.60,0.79,0.80,0.76,0.86])))
print("fixed threshold mean: {:.3f}".format(np.mean([0.62,0.79,0.78,0.76,0.88])))
print("seed 1          mean: {:.3f}".format(np.mean([0.64,0.82,0.77,0.81,0.90])))
print("seed 1 (loc)    mean: {:.3f}".format(np.mean([0.62,0.81,0.77,0.79,0.88])))


| Test Person   | Paper   | Paper Repro   | loc fix seed 1| loc fix seed 1 train | 
| ------------- |:-------:| :-----------: | :------------:|                      |
| J             | 0.624   | 0.59 (0.04)   | 0.62 (0.05)   | 0.64 (0.04)          |
| Ni            | 0.791   | 0.74 (0.05)   | 0.81 (0.04)   | 0.81 (0.05)          |
| S             | 0.809   | 0.77 (0.03)   | 0.77 (0.04)   | 0.80 (0.05)          |
| Na            | 0.831   | 0.79 (0.04)   | 0.79 (0.05)   | 0.80 (0.06)          |
| L             | 0.926   | 0.86 (0.03)   | 0.88 (0.05)   | 0.88 (0.05)          |
| ------------- |---------| ------------- |               |                      |
| Diff paper    |         |               |               |                      |
| Mean          | 0.796   | 0.75          |               |                      |



Results in comparison:

| Test Person   | Paper   | Paper Repro   | Opt. Treshold | Fixed Treshold | fixed seed 1   |loc fix seed 1| 
| ------------- |:-------:| :-----------: | :-----------: | :------------: | :------------: |:------------:|
| J             | 0.624   | 0.59 (0.04)   | 0.60 (0.04)   | 0.62 (0.04)    | 0.64 (0.05)    | 0.62 (0.05)  |
| Ni            | 0.791   | 0.74 (0.05)   | 0.79 (0.03)   | 0.79 (0.04)    | 0.82 (0.04)    | 0.81 (0.04)  |
| S             | 0.809   | 0.77 (0.03)   | 0.80 (0.03)   | 0.78 (0.04)    | 0.77 (0.06)    | 0.77 (0.04)  |
| Na            | 0.831   | 0.79 (0.04)   | 0.76 (0.06)   | 0.76 (0.04)    | 0.81 (0.04)    | 0.79 (0.05)  |
| L             | 0.926   | 0.86 (0.03)   | 0.86 (0.03)   | 0.88 (0.04)    | 0.90 (0.05)    | 0.88 (0.05)  |
| ------------- |---------| ------------- | ------------- | ---------------| ---------------|              |
| Diff paper    |         |               | 0.034         | 0.030          | ---------------|              |
| Mean          | 0.796   | 0.75          | 0.762         | 0.766          | 0.788          |              |

#### TODO: why is the testscore here lower than in OGER? supposed to be 0.8

In [ ]:
eval_models = {}
eval_scores = {}
for i, (testfile, scores, networks) in enumerate(zip(files, all_scores,all_networks)):
    print(i, testfile)
    # get median model
    median_idx = np.argsort(scores)[len(scores)//2]
    median_score = scores[median_idx]
    median_network = networks[median_idx]
    eval_models[testfile] = median_network
    eval_scores[testfile] = median_score
median_network

In [ ]:
import matplotlib
#matplotlib.rcParams.update({'font.size': 20})
     
fix_seed(1)

for key in eval_models.keys():
    
    trainset, testset, trainloader, testloader = createData(set(files) - set([key]), [key])
    score, accuracy, _ = testESN(eval_models[key], testloader, learnTreshold, fixed_threshold=0.4, 
                    plot=True, conf_mat_title='')

    plt.subplots_adjust(bottom=0.1)
    pp = PdfPages('figures/esn_experiment_{}_f1_score_{:.2f}.pdf'.format(key, score[0]))
    pp.savefig()
    plt.savefig('figures/esn_experiment_{}_f1_score_{:.2f}.eps'.format(key, score[0]), format='eps')

    plt.pause(0.1)
    pp.close()

In [ ]:
from Evaluation import plot_confusion_matrix

for i, (testfile, scores, cms) in enumerate(zip(files, all_scores,np.array(all_cms))):
    fig = plot_confusion_matrix(cms.mean(0), gestures=totalGestureNames[:10] + totalGestureNames[-1:])
    plt.xlim(-0.5, 10.5)
    plt.ylim(10.5, -0.5)
    fig.tight_layout(pad=3.0)
    pp = PdfPages('figures/esn_experiment_{}_avg_f1_score_{:.2f}.pdf'.format(testfile, np.mean(scores)))
    pp.savefig()
    pp.close()
    plt.savefig('figures/esn_experiment_{}_avg_f1_score_{:.2f}.eps'.format(testfile, np.mean(scores)), format='eps')


## Section 3: Meassure training time

We are using timeit to meassure training time, two repetitions of 10 loops.

In [ ]:
print("Training on: {}".format(inputFiles))

In [ ]:
%%timeit -r2 -n10 
esn = createESN()
trainESN(trainloader, esn)

# Experimental code

Code below here is experimental and not used in the paper

#### Try out softmax

Learn threshold must be true for softmax to make sense.

In [ ]:
assert learnTreshold==True, 'Model must be created with learnThreshold=true to use softmax'

for test_inputs, test_targets in testloader:
    plt.figure(figsize=(20,5))
    outputs = esn(test_inputs.float())
    outputs = nn.Softmax(2)(outputs)
    if learnTreshold:
        plt.plot(outputs[0,:800,10], c='black')
    plt.plot(outputs[0,:800,:10])
    plt.plot(test_targets[0,:800,:])
plt.tight_layout()

AssertionError: ignored

In [ ]:

def calcBasicMaxActivation(prediction, t_target, threshold, gestureMinLength=1 ):
    inactive_timesteps = prediction.max(1) < threshold
    active_timesteps = prediction.max(1) >= threshold

    gesture_end = inactive_timesteps[1:] * active_timesteps[:-1]
    gesture_start = inactive_timesteps[:-1] * active_timesteps[1:]

    predicted_labels = np.zeros(prediction.shape)
    for start, end in list(zip(np.where(gesture_start)[0], np.where(gesture_end)[0])):

        if end-start > gestureMinLength:
            gestureclass = prediction[start:end].sum(0).argmax()
            predicted_labels[start:end,gestureclass] = 1
    return predicted_labels


In [ ]:
import sklearn
testCms = []
testF1MaxApps = []

t_target =  test_targets[0].numpy()
prediction = outputs[0].numpy()[:,:10]
if learnTreshold: # if threshold is learned, then it's the las collumn of the prediction
    threshold = outputs[0].numpy()[:,10]
else: #else add a constant threshold
    threshold = np.ones((prediction.shape[0],1))*0.4

# use different method for maxappactivation
t_maxApp_prediction = calcBasicMaxActivation(prediction,t_target,threshold, 10)


pred_MaxApp, targ_MaxApp = Evaluation.calcInputSegmentSeries(t_maxApp_prediction, t_target, 0.5)
testF1MaxApps.append(np.mean(sklearn.metrics.f1_score(targ_MaxApp,pred_MaxApp,average=None)))


#print(t_maxApp_prediction.shape, prediction.shape, pred_MaxApp, targ_MaxApp)
plt.figure(figsize=(20,3))
plt.plot(t_maxApp_prediction[:800])
plt.plot(t_target[:800])


conf = sklearn.metrics.confusion_matrix(targ_MaxApp, pred_MaxApp)
testCms.append(conf)

Evaluation.plot_confusion_matrix(testCms[0], gestureNames, 'test set')
plt.tight_layout()
plt.ylim(10.5,-0.5)

print("Test f1 score for maxactivity: {:.2f}, reg: {:.2f}".format(testF1MaxApps[0], trainF1s[0]))




ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-49a8ed43a2e5>", line 5, in <module>
    t_target =  test_targets[0].numpy()
NameError: name 'test_targets' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.7/dist-packages/IPyth

NameError: ignored